In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O 
import os

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import re

import warnings
warnings.filterwarnings('ignore')

/mnt/c/Users/denad/OneDrive/Documents/GitHub/uefa-championship-analysis-1/.venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:545: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


### Data Loading

In [3]:
#============== Concat all matches over the years into one file ===============#
# Path to the directory containing the yearly matches CSV files
data_dir = '../inputs/matches/euro/'

# List all CSV files in the directory
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

all_matches = pd.DataFrame()

for file in csv_files:
    # Extract the year from the filename (format 'YYYY.csv')
    year = int(file.split('.')[0])
    df = pd.read_csv(os.path.join(data_dir, file))
    
    # Add a new column 'year' to the DataFrame
    df['year'] = year
    
    # Append the DataFrame to the combined DataFrame
    all_matches = pd.concat([all_matches, df], ignore_index=True)

all_matches.head()

,id_match,home_team,away_team,home_team_code,away_team_code,home_score,away_score,home_penalty,away_penalty,home_score_total,...,penalties_missed,penalties,red_cards,game_referees,stadium_city,stadium_name,stadium_name_media,stadium_name_official,stadium_name_event,stadium_name_sponsor
0,4025,USSR,Yugoslavia,URS,YUG,1.0,1.0,NaN,NaN,2.0,...,NaN,NaN,NaN,[],Paris,Parc des Princes,Parc des Princes,Parc des Princes,Parc des Princes,Parc des Princes
1,4024,Czechoslovakia,France,TCH,FRA,2.0,0.0,NaN,NaN,2.0,...,NaN,NaN,NaN,[],Marseille,Stade de Marseille,Stade de Marseille,Stade de Marseille,Stade de Marseille,Orange Vélodrome
2,4023,Czechoslovakia,USSR,TCH,URS,0.0,3.0,NaN,NaN,0.0,...,NaN,NaN,NaN,[],Marseille,Stade de Marseille,Stade de Marseille,Stade de Marseille,Stade de Marseille,Orange Vélodrome
3,4022,France,Yugoslavia,FRA,YUG,4.0,5.0,NaN,NaN,4.0,...,NaN,NaN,NaN,[],Paris,Parc des Princes,Parc des Princes,Parc des Princes,Parc des Princes,Parc des Princes
4,3996,Spain,USSR,ESP,URS,2.0,1.0,NaN,NaN,2.0,...,NaN,NaN,NaN,[],Madrid,Estadio Santiago Bernabéu,Estadio Santiago Bernabéu,Estadio Santiago Bernabéu,Estadio Santiago Bernabéu,NaN


In [4]:
all_matches.shape

(388, 47)

In [ ]:
all_matches.info()

### Data Preprocessing

In [5]:
#====== drop all matches that have not taken place yet (referring to 2024) =====#
all_matches = all_matches.loc[all_matches['status'] == 'FINISHED']

all_matches.to_csv('../data/all_matches.csv', index=False)
all_matches.shape

(350, 47)

### Participants

In [ ]:
# # Function to load country logos
# def get_image(path):
#     return OffsetImage(plt.imread(path), zoom=0.04)

# # Function to plot heatmap with logos
# def plot_with_logos(data, title, size=(16, 16), is_save=False):
#     '''
#     Plot heatmap with country logos.
#     '''
#     dpi = 200
#     fig, ax = plt.subplots(figsize=size, dpi=dpi)
    
#     # Create a mask for cells without logos
#     mask = data.copy()
#     mask[:] = False
    
    
#     # Customize ticks and labels
#     ax.tick_params(axis='x', labelsize=14)
#     ax.tick_params(axis='y', labelsize=14)
#     ax.set_ylabel('')
#     ax.set_xlabel('')
#     ax.xaxis.tick_top()
#     ax.xaxis.set_ticks_position('none')
#     ax.yaxis.set_ticks_position('none') 
#     ax.spines[['bottom', 'right']].set_visible(True)
#     ax.set_title(title, size=22)
    
#     # Add country logos and update the mask
#     for (i, j), val in np.ndenumerate(data):
#         if val == 1:
#             # Extract the country code using regex
#             country_code = re.search(r'\((.*?)\)', data.index[i]).group(1)
#             try:
#                 logo_path = f'../inputs/logos/{country_code}.png'
#                 ab = AnnotationBbox(get_image(logo_path), (j + 0.5, i + 0.5), frameon=False)
#                 ax.add_artist(ab)
#             except FileNotFoundError:
#                 mask.iloc[i, j] = True  # Mark cell to be colored black
                
#     # Plot the heatmap with the mask
#     sns.heatmap(data, annot=False, cbar=False, linewidths=0.5, 
#                 linecolor='black', square=True, cmap='Spectral', ax=ax, mask=~mask, 
#                 facecolor='black')

#     plt.tight_layout()
#     if is_save:
#         plt.savefig(f'../data/{title.lower()}.png')
#     plt.show()

In [ ]:
# # Group matches by year and aggregate teams
# all_matches['home_team_extra'] = all_matches.apply(lambda row: f"{row['home_team']} ({row['home_team_code']})", axis=1)
# all_matches['away_team_extra'] = all_matches.apply(lambda row: f"{row['away_team']} ({row['away_team_code']})", axis=1)
# df_team = all_matches.groupby('year').agg(home=('home_team_extra', list), away=('away_team_extra', list))
# df_team['teams'] = df_team.apply(lambda x: list(set(x['home'] + x['away'])), axis=1)
# df_team['#'] = df_team['teams'].apply(len)

# # Pivot table for heatmap
# df_team_pivot = df_team['teams'].explode().reset_index().pivot(index='year', columns='teams', values='year').T
# df_team_pivot[df_team_pivot > 0] = 1
# column_list = sorted(df_team.index, reverse=True)

# df_team_pivot['times'] = df_team_pivot.sum(axis=1)
# df_team_pivot = df_team_pivot.sort_values(['times'] + column_list, ascending=[False] * (len(column_list) + 1))

# df_team_pivot.columns = [str(col)[:-2] + '\n' + str(col)[-2:] for col in column_list[::-1]] + ['times']
# df_team_pivot = df_team_pivot.drop('times', axis=1)
# df_team_pivot = df_team_pivot.sort_index()

# # Title for the plot
# title = 'UEFA EURO Cup Participants (1960 - 2024)'

# # Plotting the heatmap with logos
# plot_with_logos(df_team_pivot, title=title, size=(12, 32), is_save=True)

Credit for heatmap: https://www.kaggle.com/code/piterfm/football-soccer-world-cup-data-analysis-wiz?scriptVersionId=111502552&cellId=7

#### Note on Historical Countries in UEFA EURO Data (1960 - 2024)

The data spans from 1960 to 2024 and includes some countries that no longer exist (Find out more [here](https://en.wikipedia.org/wiki/UEFA)). Below is the information about these historical countries and the modern countries they have become:

1. **CIS (Commonwealth of Independent States)**
   - **Existed:** 1992
   - **Countries:** Armenia 🇦🇲, Azerbaijan 🇦🇿, Belarus 🇧🇾, Georgia 🇬🇪, Kazakhstan 🇰🇿, Kyrgyzstan 🇰🇬, Moldova 🇲🇩, Russia 🇷🇺, Tajikistan 🇹🇯, Turkmenistan 🇹🇲, Ukraine 🇺🇦, Uzbekistan 🇺🇿

2. **USSR (Union of Soviet Socialist Republics)**
   - **Existed:** 1922–1991
   - **Countries:** Armenia 🇦🇲, Azerbaijan 🇦🇿, Belarus 🇧🇾, Estonia 🇪🇪, Georgia 🇬🇪, Kazakhstan 🇰🇿, Kyrgyzstan 🇰🇬, Latvia 🇱🇻, Lithuania 🇱🇹, Moldova 🇲🇩, Russia 🇷🇺, Tajikistan 🇹🇯, Turkmenistan 🇹🇲, Ukraine 🇺🇦, Uzbekistan 🇺🇿

3. **West Germany (Federal Republic of Germany)**
   - **Existed:** 1949–1990
   - **Countries:** Part of the current unified Germany 🇩🇪

4. **Yugoslavia**
   - **Existed:** 1918–1992 (as the Kingdom of Yugoslavia and later the Socialist Federal Republic of Yugoslavia)
   - **Countries:** Bosnia and Herzegovina 🇧🇦, Croatia 🇭🇷, Macedonia 🇲🇰, Montenegro 🇲🇪, Serbia 🇷🇸, Slovenia 🇸🇮

5. **Czechoslovakia**
   - **Existed:** 1918–1992
   - **Countries:** Czech Republic 🇨🇿, Slovakia 🇸🇰

### Goal Trend Analysis over years


In [ ]:
one_match = all_matches.loc[all_matches['id_match'] == 2024491]
# one_match['home_team']
one_match['red_cards'].iloc[0]

In [ ]:
all_matches.loc[all_matches['red_cards'].notnull()]['red_cards'].iloc[0]

In [6]:
all_matches.info()

<class 'pandas.core.frame.DataFrame'>
Index: 350 entries, 0 to 387
Data columns (total 47 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_match               350 non-null    int64  
 1   home_team              350 non-null    object 
 2   away_team              350 non-null    object 
 3   home_team_code         350 non-null    object 
 4   away_team_code         350 non-null    object 
 5   home_score             350 non-null    float64
 6   away_score             350 non-null    float64
 7   home_penalty           22 non-null     float64
 8   away_penalty           22 non-null     float64
 9   home_score_total       350 non-null    float64
 10  away_score_total       350 non-null    float64
 11  winner                 285 non-null    object 
 12  winner_reason          350 non-null    object 
 13  year                   350 non-null    int64  
 14  date                   350 non-null    object 
 15  date_time  

### Extract Tournament Winners Data

In [25]:
# Extract final match details for each year
finals = all_matches[all_matches['round'] == 'FINAL']

tournament_winners_list = []
for year, group in finals.groupby('year'):
    winner = group['winner'].values[0]
    runner_up = group['away_team'].values[0] if group['winner'].values[0] == group['home_team'].values[0] else group['home_team'].values[0]
    score = f"{int(group['home_score'].values[0])}-{int(group['away_score'].values[0])}"
    participants = pd.concat([all_matches[all_matches['year'] == year]['home_team'], 
                              all_matches[all_matches['year'] == year]['away_team']]).unique()
    total_participants = len(participants)
    
    tournament_winners_list.append({
        'year': year,
        'totalParticipants': total_participants,
        'winner': winner,
        'runnerUp': runner_up,
        'score': score
    })

tournament_winners = pd.DataFrame(tournament_winners_list)
tournament_winners.to_csv('../data/tournament_winners.csv', index=False)

### Extract First-time Participations for each country

In [28]:
# Extract first participation year for each country
home_teams = all_matches[['year', 'home_team', 'home_team_code']].drop_duplicates()
away_teams = all_matches[['year', 'away_team', 'away_team_code']].drop_duplicates()

teams = pd.concat([home_teams.rename(columns={'home_team': 'team', 'home_team_code': 'team_code'}),
                   away_teams.rename(columns={'away_team': 'team', 'away_team_code': 'team_code'})])

first_participation = teams.groupby('team').year.min().reset_index()
first_participation = first_participation.sort_values('year')
first_participation = first_participation.merge(teams[['team', 'team_code']].drop_duplicates(), on='team')

# Calculate total participations, total wins, and final appearances
total_participations = teams.groupby('team').size().reset_index(name='total_participations')
total_wins = finals['winner'].value_counts().reset_index(name='total_wins').rename(columns={'winner': 'team'})
final_appearances = pd.concat([finals['home_team'], finals['away_team']]).value_counts().reset_index(name='final_appearances').rename(columns={'index': 'team'})

# Merge the new fields with the first participation data
first_participation = first_participation.merge(total_participations, on='team', how='left')
first_participation = first_participation.merge(total_wins, on='team', how='left')
first_participation = first_participation.merge(final_appearances, on='team', how='left')

# Fill NaN values with 0 for total_wins and final_appearances
first_participation['total_wins'] = first_participation['total_wins'].fillna(0).astype(int)
first_participation['final_appearances'] = first_participation['final_appearances'].fillna(0).astype(int)

first_participation.to_csv('../data/first_participation.csv', index=False)
first_participation.head()

,team,year,team_code,total_participations,total_wins,final_appearances
0,Czechoslovakia,1960,TCH,4,1,1
1,France,1960,FRA,20,2,3
2,Yugoslavia,1960,YUG,8,0,3
3,USSR,1960,URS,7,1,4
4,Denmark,1964,DEN,19,1,1
